# Архитектура RNN. Классификация текста.

Привет! В это семинаре мы познакомимся с задачей классификации текста, на примере поиска тематики новости, а также с двумя из основных архитектурт рекуррентных нейросетей – RNN и GRU.

Нам потребуется одна библиотека от `HuggingFace🤗` под названием `datasets`. Она содержит большое число датасетов, которые используются в NLP.

In [ ]:
#pip install datasets

In [ ]:
import random
import numpy as np

import nltk
import gensim.downloader as api

import torch
import torch.nn as nn
import datasets

In [ ]:
# За детерминизм!
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

Загрузим датасет новостей: `AgNews`. В нем разделены тексты на 4 темы: `World`, `Sports`, `Business`, `Sci/Tech`. Посмотрим на структуру датасета и на примеры текстов:

In [ ]:
dataset = datasets.load_dataset("ag_news")
dataset["train"]

In [ ]:
dataset["train"][0]

В `dataset` находятся `train` и `test` части датасета.

In [ ]:
dataset

Чтобы превращать текст из набора слов в набор векторов мы будем использовать предобученные эмбеддинги. Посмотрим на их список и выберем один из них.

In [ ]:
print("\n".join(api.info()['models'].keys()))

In [ ]:
word2vec = api.load("glove-twitter-50")

Объеденим `dataset`, `word2vec` в один объект, который будет возвращать массив из векторов-эмбеддингов и класс текста.

In [ ]:
from torch.utils.data import Dataset


class AgNewsDataset(Dataset):
    def __init__(self, word2vec, train=True, max_length=128):
        self.data = dataset["train"] if train else dataset["test"]
        self.tokenizer = nltk.WordPunctTokenizer()
        self.word2vec = word2vec
        self.max_length = max_length # Лучше ограничить длину текстов
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[item]["text"]
        tokens = self.tokenizer.tokenize(text.lower())
        embeds = [
            self.word2vec.get_vector(token) 
            for token in tokens if token in self.word2vec
        ][:self.max_length]
        return {"inputs": (np.array(embeds) - self.mean) / self.std, "label": self.data[item]["label"]}

    def __len__(self):
        return len(self.data)

Обычно для сбора объектов в батчи просто собирают фиксированное число объектов. В случае текстов это не работает – каждый текст имеет произвольный размер. Можно дополнить предложения нулями, но это не эффективно. А можно собирать тексты в батчи на основе их длин. Тогда будем ограничивать число токенов в одном батче, и пока это число не было превышено, добавлять новые тексты в батч.

Для более эффективного распределения ресурсов надо рассортировать тексты по длинам. Но простая реализация этого не позволяет обучать модель эффективно(батчи не будут случайны). Поэтому на семинаре рассмотрим более простую реализацию.

In [ ]:
from torch.utils.data import Sampler, RandomSampler


class TextSampler(Sampler):
    def __init__(self, sampler, batch_size_tokens=1e5):
        self.sampler = sampler
        self.batch_size_tokens = batch_size_tokens

    def __iter__(self):
        batch = []
        max_len = 0
        for ix in self.sampler:
            row = self.sampler.data_source[ix]
            max_len = max(max_len, len(row["inputs"]))
            if (len(batch) + 1) * max_len > self.batch_size_tokens:
                yield batch
                batch = []
                max_len = len(row["inputs"])
            batch.append(ix)
        if len(batch) > 0:
            yield batch

    def __len__(self):
        return len(self.sampler)


In [ ]:
def collate_fn(batch):
    max_len = max(len(row["inputs"]) for row in batch)
    input_embeds = np.zeros((len(batch), max_len, word2vec.vector_size))
    labels = np.zeros((len(batch),))
    for idx, row in enumerate(batch):
        input_embeds[idx][:len(row["inputs"])] += row["inputs"]
        labels[idx] = row["label"]
    return {"inputs": torch.FloatTensor(input_embeds), "labels": torch.LongTensor(labels)}

In [ ]:
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler


train_dataset = AgNewsDataset(word2vec, train=True)
valid_dataset = AgNewsDataset(word2vec, train=False)
train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)

train_loader = DataLoader(train_dataset, batch_sampler=TextSampler(train_sampler), collate_fn=collate_fn, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_sampler=TextSampler(valid_sampler), collate_fn=collate_fn, num_workers=4)

## CNN

Первая модель, которую мы рассмотрим: CNN. Одномерная конволюция достаточно хорошо справляется с задачей классификации. В конце надо собрать вектор текста с помощью `AdaptiveMaxPool1d` или `AdiptiveAvgPool1d`. Для классиффикации можно собрать любую Feed Forward Network.

In [ ]:
class CNNModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()

        self.cnn = nn.Sequential(
            nn.Conv1d(embed_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.Conv1d(hidden_size, hidden_size, kernel_size=3, padding=1, stride=2),
            nn.BatchNorm1d(hidden_size),
            nn.ReLU(),
            nn.AdaptiveMaxPool1d(1),
            nn.Flatten(),
        )
        self.cl = nn.Sequential(
            nn.Linear(hidden_size, num_classes)
        )

    def forward(self, x):
        x = x.permute(0, 2, 1)
        features = self.cnn(x)
        prediction = self.cl(features)
        return prediction

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = CNNModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1

Подготовим функцию для обучения модели:

In [ ]:
import time
from tqdm.notebook import tqdm


def training(model, criterion, optimizer, num_epochs, max_grad_norm=0):
    for e in range(num_epochs):
        model.train()
        num_iter = 0
        pbar = tqdm(train_loader)
        for batch in pbar:
            input_embeds = batch["inputs"].to(device)
            labels = batch["labels"].to(device)

            optimizer.zero_grad()
            prediction = model(input_embeds)
            loss = criterion(prediction, labels)
            loss.backward()
            pbar.update(labels.size(0))
            if max_grad_norm is not None:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
            num_iter += 1
        valid_loss = 0
        valid_acc = 0
        num_iter = 0
        model.eval()
        with torch.no_grad():
            for batch in valid_loader:
                input_embeds = batch["inputs"].to(device)
                labels = batch["labels"].to(device)

                prediction = model(input_embeds)
                valid_loss += criterion(prediction, labels)
                valid_acc += (labels == torch.max(prediction, axis=1)[1]).float().mean()
                num_iter += 1

        print(f"Valid Loss: {valid_loss / num_iter}, accuracy: {valid_acc / num_iter}")

In [ ]:
training(model, criterion, optimizer, num_epochs)

## RNN

Вторая модель: RNN. Это рекуррентная сеть, она использует скрытое состояние из прошлой иттерации для создания нового. Это описывается с помощью формул:

$$
h_t = \tanh(W_{ih} x_t + b_{ih} + W_{hh} h_{(t-1)} + b_{hh})
$$

Напишем этот модуль на `Torch`!



In [ ]:
class RNN(nn.Module):
    def __init__(self, embed_size, hidden_size):
        super().__init__()

        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.w_h = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_h = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_x = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_x = nn.Parameter(torch.rand(1, hidden_size))

    def forward(self, x, hidden = None):
        '''
        x – torch.FloatTensor with the shape (bs, *, emb_size)
        hidden - torch.FloatTensro with the shape (bs, hidden_size)
        return: torch.FloatTensor with the shape (bs, hidden_size)
        '''
        hidden = torch.zeros((x.size(0), self.hidden_size)).to(x.device) if hidden is None else hidden
        for idx in range(x.size(1)):
            hidden = torch.tanh(x[:, idx] @ self.w_x + self.b_x + hidden @ self.w_h + self.b_h)
        return hidden

In [ ]:
class RNNModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()

        self.rnn = RNN(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        hidden = self.rnn(x)
        output = self.cls(hidden)
        return output

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = RNNModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [ ]:
training(model, criterion, optimizer, num_epochs, max_grad_norm)

## GRU

Третья модель: GRU. Она усложненная версия `RNN`. Гланая идея GRU: гейты. Так реализуется "память" модели – она маскирует часть старого скрытого состояния, создавая на этом месте новое. Модель GRU описывается следующим образом:

$$
\begin{array}{ll}
            r_t = \sigma(W_{ir} x_t + b_{ir} + W_{hr} h_{(t-1)} + b_{hr}) \\
            z_t = \sigma(W_{iz} x_t + b_{iz} + W_{hz} h_{(t-1)} + b_{hz}) \\
            n_t = \tanh(W_{in} x_t + b_{in} + r_t * (W_{hn} h_{(t-1)}+ b_{hn})) \\
            h_t = (1 - z_t) * n_t + z_t * h_{(t-1)}
        \end{array}
$$

In [ ]:
class GRU(nn.Module):
    def __init__(self, embed_size, hidden_size):
        super().__init__()

        self.embed_size = embed_size
        self.hidden_size = hidden_size

        self.w_rh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_rh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_rx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_rx = nn.Parameter(torch.rand(1, hidden_size))

        self.w_zh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_zh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_zx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_zx = nn.Parameter(torch.rand(1, hidden_size))

        self.w_nh = nn.Parameter(torch.rand(hidden_size, hidden_size))
        self.b_nh = nn.Parameter(torch.rand((1, hidden_size)))
        self.w_nx = nn.Parameter(torch.rand(embed_size, hidden_size))
        self.b_nx = nn.Parameter(torch.rand(1, hidden_size))

    def forward(self, x, hidden = None):
        '''
        x – torch.FloatTensor with the shape (bs, *, emb_size)
        hidden - torch.FloatTensro with the shape (bs, hidden_size)
        return: torch.FloatTensor with the shape (bs, hidden_size)
        '''
        hidden = torch.zeros((x.size(0), self.hidden_size)).to(x.device) if hidden is None else hidden
        for idx in range(x.size(1)):
            r = torch.sigmoid(x[:, idx] @ self.w_rx + self.b_rx + hidden @ self.w_rh + self.b_rh)
            z = torch.sigmoid(x[:, idx] @ self.w_zx + self.b_zx + hidden @ self.w_zh + self.b_zh)
            n = torch.tanh(x[:, idx] @ self.w_zx + self.b_zx + r * (hidden @ self.w_zh + self.b_zh))
            hidden = (1 - z) * n + z * hidden
        return hidden

In [ ]:
class GRUModel(nn.Module):
    def __init__(self, embed_size, hidden_size, num_classes=4):
        super().__init__()

        self.gru = GRU(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        hidden = self.gru(x)
        output = self.cls(hidden)
        return output

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRUModel(word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [ ]:
training(model, criterion, optimizer, num_epochs, max_grad_norm)

## GRU + Embeddings

Пока что мы фиксировали эмбеддинги в наших моделях. Но их можно доучить на наших данных! Для этого надо немного переделать способ подачи данных в модель и добавить в модель модуль `Embedding`. По-экспериментируем на модели `GRU`.

In [ ]:
class AgNewsDatasetv2(Dataset):
    def __init__(self, train=True, max_length=128):
        self.data = dataset["train"] if train else dataset["test"]
        self.tokenizer = nltk.WordPunctTokenizer()
        self.max_length = max_length
        self.vocab = set(
            self.tokenizer.tokenize("".join(d["text"].lower() for d in self.data))
        )
        self.word2idx = {word: idx + 1 for idx, word in enumerate(self.vocab)}

    def __getitem__(self, item):
        text = self.data[item]["text"]
        tokens = self.tokenizer.tokenize(text.lower())
        embeds = [self.word2idx.get(token, 0) for token in tokens][:self.max_length]
        return {"inputs": embeds, "label": self.data[item]["label"]}

    def __len__(self):
        return len(self.data)

In [ ]:
def collate_fn_v2(batch):
    max_len = max(len(row["inputs"]) for row in batch)
    input_embeds = np.zeros((len(batch), max_len))
    labels = np.zeros((len(batch),))
    for idx, row in enumerate(batch):
        input_embeds[idx][:len(row["inputs"])] += row["inputs"]
        labels[idx] = row["label"]
    return {"inputs": torch.LongTensor(input_embeds), "labels": torch.LongTensor(labels)}

In [ ]:
train_dataset = AgNewsDatasetv2(train=True)
valid_dataset = AgNewsDatasetv2(train=False)
valid_dataset.vocab = train_dataset.vocab
valid_dataset.word2idx = train_dataset.word2idx
train_sampler = RandomSampler(train_dataset)
valid_sampler = SequentialSampler(valid_dataset)

train_loader = DataLoader(train_dataset, batch_sampler=TextSampler(train_sampler), collate_fn=collate_fn_v2, num_workers=4)
valid_loader = DataLoader(valid_dataset, batch_sampler=TextSampler(valid_sampler), collate_fn=collate_fn_v2, num_workers=4)

In [ ]:
class GRUModelv2(nn.Module):
    def __init__(self, voc_size, embed_size, hidden_size, num_classes=4):
        super().__init__()

        self.emb = nn.Embedding(voc_size, embed_size)
        self.gru = GRU(embed_size, hidden_size)
        self.cls = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        x = self.emb(x)
        hidden = self.gru(x)
        output = self.cls(hidden)
        return output

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = GRUModelv2(len(train_dataset.vocab) + 1, word2vec.vector_size, 50).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

num_epochs = 1
max_grad_norm = 1.0

In [ ]:
with torch.no_grad():
    for word in train_dataset.vocab:
        if word in word2vec:
            model.emb.weight[train_dataset.word2idx[word]] = torch.from_numpy(word2vec.get_vector(word))

In [ ]:
training(model, criterion, optimizer, num_epochs, max_grad_norm)